In [7]:
from datetime import datetime
from satlomas.configuration import LSTMTrainingScriptConfig

from satlomas.data import read_time_series_from_csv
from satlomas.feature import (
    get_dataset_from_series,
    get_interest_variable
)
from satlomas.model_hyperopt import (
    get_lstm_nnet_opt
)
from satlomas.model import (train_val_test_split)


from hyperopt import tpe, hp, fmin



In [13]:
config_file = '../config_train_lstm_temp.json'
script_config = LSTMTrainingScriptConfig(config_file)

n_past_steps = script_config.n_past_steps
input_csv = script_config.input_csv

date_col = script_config.date_col
hr_col = script_config.hr_col
target_var = script_config.numeric_var
sensor_var = script_config.sensor_var
sensor = script_config.target_sensor
output_models_path = script_config.output_models_path
output_results_path = script_config.output_results_path

base_config = script_config.base_config
mid_layers_config = script_config.mid_layers_config
model_loss = script_config.model_loss
optimizer = script_config.optimizer

early_stop_patience=script_config.early_stop_patience
epochs=script_config.epochs

# read the raw data
input_csv_nb = '../{}'.format(input_csv)
raw_dataset = read_time_series_from_csv(input_csv_nb,date_col,hr_col,target_var,sensor_var)
raw_dataset.head()

# get the time series dataset
time_series_dset = get_interest_variable(raw_dataset,sensor_var,date_col,hr_col,target_var,sensor)
time_series_dset.head()

# get the final dataset
sup_dataset,scaler = get_dataset_from_series(time_series_dset,n_past_steps)
print(scaler)
sup_dataset.head()

# split the dataset in train , test and validation
n_features = time_series_dset.shape[1]
dataset_splits = train_val_test_split(sup_dataset,n_past_steps,n_features,target_var)


model_loss = script_config.model_loss
optimizer = script_config.optimizer

early_stop_patience=script_config.early_stop_patience
epochs=script_config.epochs

time_stmp = datetime.now()
 
time_stmp_str = time_stmp.strftime("%Y-%m-%d_%H:%M:%S")

# read the raw data
output_models_path = '../{}'.format(output_models_path)

out_model_name = '{}{}_hyperopt_model_{}.hdf5'.format(
        output_models_path,
        str(script_config),
        time_stmp_str)

history_out_name = '{}{}_hyperopt_history_{}.pickle'.format(
        output_models_path,
        str(script_config),
        time_stmp_str)


print(out_model_name)

space = hp.choice('nnet_config',[
        {'dataset_splits': dataset_splits,
         'mult': hp.choice('mult',[2,3]),
         'dropout_rate' : hp.uniform('dropout_rate', 0.1, 0.7),
         'n_mid_layers': 2,
         'model_loss' : model_loss ,
         'optimizer' : optimizer,
         'sensor' :sensor,
         'target_var' : target_var,
         'output_models_path' : output_models_path,
         'early_stop_patience': early_stop_patience,
         'epochs' : epochs,
         'time_stmp_str' : time_stmp_str,
         'out_model_name' : out_model_name,
         'history_out_name' : history_out_name
        }
        ])

read dataset of shape (160584, 3)
MinMaxScaler(copy=True, feature_range=(0, 1))
Sampleamos datasets de futuro a pasado
(96348, 3) 96348 (96348,)
(96348, 3, 1) (96348,) (32116, 3, 1) (32116,) (32117, 3, 1) (32117,)
../models/esp:10_eps:2_loss:mean_squared_error_opt:adam_pstps:3_sensor:A620_var:temp_basenet:4.4_midnet:4.2_hyperopt_model_2020-02-24_16:58:53.hdf5


In [ ]:

best_nnet = fmin(get_lstm_nnet_opt,space,algo=tpe.suggest,max_evals=3)
print(best_nnet)


Train on 96348 samples, validate on 32116 samples
Epoch 1/2
   32/96348 [..............................]
 - ETA: 1:29:26 - loss: 0.3652
                                                    
  160/96348 [..............................]
 - ETA: 18:23 - loss: 0.3891  
                                                    
  224/96348 [..............................]
 - ETA: 13:29 - loss: 0.3761
                                                    
  288/96348 [..............................]
 - ETA: 10:47 - loss: 0.3755
                                                    
  320/96348 [..............................]
 - ETA: 10:04 - loss: 0.3729
                                                    
  384/96348 [..............................]
 - ETA: 8:37 - loss: 0.3697 
                                                     
  448/96348 [..............................]
 - ETA: 7:34 - loss: 0.3709
                                                     
  512/96348 [..............................]
 